# Vowpal Wabbit в NLP
Автоматическая обработка текстов - 2017, семинар 4.

В этом семинаре мы познакомимся с библиотекой Vowpal Wabbit и решим с его помощью задачу многоклассовой классификации на больших данных. 
Данные скачайте [здесь](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data) или запустите следующие две ячейки.

In [1]:
# ! wget https://www.dropbox.com/s/r0q0p0uprhcp8bb/train-sample.zip
# ! unzip train-sample.zip

In [2]:
# ! wget https://www.dropbox.com/s/50vw2gsglc91f6o/train.zip
# ! unzip train.zip

Чтобы на семинаре не тратить время на обработку и обучение моделей на всех данных, предлагается использовать только небольшую подвыборку (`train-sample.csv`). Но сдавать ноутбук все равно необходимо с результатами на **всех** данных.

In [116]:
import csv

INPUT_DATA = 'train.csv'
# INPUT_DATA = 'train-sample.csv'

reader = csv.DictReader(open(INPUT_DATA))
dict(next(reader))

{'BodyMarkdown': "I'm new to C#, and I want to use a trackbar for the forms opacity\nThis is my code\n\n    decimal trans = trackBar1.Value / 5000\n    this.Opacity = trans\n\nWhen I try to build it, I get this error\n\n**Cannot implicitly convert type 'decimal' to 'double**\n\nI tried making trans a double, but then the control doesn't work. This code worked fine for me in VB.NET. Any suggestions?",
 'OpenStatus': 'open',
 'OwnerCreationDate': '07/31/2008 21:33:24',
 'OwnerUndeletedAnswerCountAtPostTime': '0',
 'OwnerUserId': '8',
 'PostClosedDate': '',
 'PostCreationDate': '07/31/2008 21:42:52',
 'PostId': '4',
 'ReputationAtPostCreation': '1',
 'Tag1': 'c#',
 'Tag2': '',
 'Tag3': '',
 'Tag4': '',
 'Tag5': '',
 'Title': 'Decimal vs Double?'}

Каждый объект выборки соответствует некоторому посту на Stack Overflow. Требуется построить модель, определяющую статус поста. Подробнее про задачу и формат данных можно прочитать на [странице соревнования](https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow).

Перед обучением модели из Vowpal Wabbit данные следует сохранить в специальный формат: <br>
`label |namespace1 feature1:value1 feature2 feature3:value3 |namespace2 ...` <br>
Записи `feature` и `feature:1.0` эквивалентны. Выделение признаков в смысловые подгруппы (namespaces) позволяет создавать взаимодействия между ними. Подробнее про формат входных данных можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Input-format).

Ниже реализована функция, которая извлекает признаки с помощью подаваемого на вход экстрактора, разбивает данные на трейн и тест и записывает их на диск.

In [117]:
from tqdm import tqdm_notebook

STATUSES = ['not a real question', 'not constructive', 'off topic', 'open', 'too localized']
STATUS_DICT = {status: i+1 for i, status in enumerate(STATUSES)}

def data2vw(features_extractor, train_output='train', test_output='test', ytest_output='ytest'):
    reader = csv.DictReader(open(INPUT_DATA, encoding='utf-8'))
    writer_train = open(train_output, 'w')
    writer_test = open(test_output, 'w')
    writer_ytest = open(ytest_output, 'w')
    
    for row in tqdm_notebook(reader):
        label = STATUS_DICT[row['OpenStatus']]
        features = features_extractor(row)
        output_line = str(label) + " "
        for namespace, words in features.items():
            output_line += "|" + namespace + " "
            output_line += words + " "
        output_line += "\n"
        if int(row['PostId']) % 2 == 0:
            writer_train.write(output_line)
        else:
            writer_test.write(output_line)
            writer_ytest.write('%s\n' % label)
            
    writer_train.close()
    writer_test.close()
    writer_ytest.close()

Начнем с простейшей модели. В качестве признаков возьмите заголовки и очистите их: приведите символы к нижнему регистру, удалите пунктуацию. Также приветствуется использование стеммеров/лемматизаторов, однако учтите, что они могут сильно замедлить скорость обработки.

In [118]:
import re
import string
import math
import numpy as np

In [119]:
def extract_title(row):
    title = row['Title']
    title = title.lower()
    translator = str.maketrans('', '', string.punctuation)
    title = title.translate(translator)
    title = re.split('\s', title)
    return ' '.join(title)

data2vw(lambda row: {'t': extract_title(row)})

! head -n 5 train


4 |t decimal vs double 
4 |t percentage width child in absolutely positioned parent doesnt work in ie7 
4 |t tools for porting j code to c 
4 |t throw error in mysql trigger 
4 |t whats the difference between mathfloor and mathtruncate 


Обучим `vw` модель. Параметр `-d` отвечает за путь к обучающей выборке, `-f` – за путь к модели, `--oaa` – за режим мультиклассовой классификации `one-against-all`. Подробное описание всех параметров можно найти [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Command-line-arguments) или вызвать `vw --help`.

In [120]:
! vw -d train --loss_function logistic --oaa 5 -f model

final_regressor = model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = train
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4        1        4
0.500000 0.000000            2            2.0        4        4       12
0.250000 0.000000            4            4.0        4        4        6
0.125000 0.000000            8            8.0        4        4        7
0.062500 0.000000           16           16.0        4        4        4
0.062500 0.062500           32           32.0        4        4        9
0.046875 0.031250           64           64.0        4        4        5
0.062500 0.078125          128          128.0        4        4       11
0.089844 0.117188          256          256.0        4        4        8
0.070312 0.050781          512          512.0   

Применим модель к тестовой выборке и сохраним предсказания.

In [121]:
! vw -i model -t test -r pred

only testing
raw predictions = pred
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        4        4        9
0.000000 0.000000            2            2.0        4        4        7
0.000000 0.000000            4            4.0        4        4        8
0.125000 0.250000            8            8.0        4        4        6
0.125000 0.125000           16           16.0        4        4        7
0.156250 0.187500           32           32.0        4        4       10
0.093750 0.031250           64           64.0        4        4        8
0.117188 0.140625          128          128.0        4        4        8
0.121094 0.125000          256          256.0        4        4       16
0.105469 0.089844          512       

In [122]:
! head -n 3 pred

1:-4.28213 2:-6.08996 3:-4.31059 4:2.40051 5:-6.81148
1:-7.59211 2:-7.83689 3:-9.37969 4:5.67713 5:-8.07567
1:-5.24459 2:-6.75469 3:-7.29165 4:3.59159 5:-7.48763


Реализуйте функцию, которая вычисляет `logloss` и `accuracy`, не загружая вектора в память. Используйте `softmax`, чтобы получить вероятности.

In [123]:
def get_scores(ytest_input='ytest', pred_input='pred'):
    n, error, loss = 0, 0, 0
    reader_ytest = open(ytest_input, 'r')
    reader_pred = open(pred_input, 'r')
    
    for label, pred in zip(reader_ytest, reader_pred):
        probs = []
        classes = pred[:-1].split(' ')
        for cl in classes:
            pred_label, pr_label = cl.split(':')
            pr_label = float(pr_label)
            probs.append(math.exp(pr_label))
        probs = np.array(probs) / np.array(probs).sum()
        
        max_prob_arg = np.argmax(probs) + 1
        max_prob = np.max(probs)
        
        #accuracy
        if int(label) != max_prob_arg:
            error = error + 1
        n = n + 1
        
        #logloss
        loss = loss + math.log(probs[int(label) - 1])
        
    reader_ytest.close()
    reader_pred.close()
    return - loss / n, 1 - float(error) / n

In [124]:
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.14763
accuracy = 0.97785


На оригинальных данных `logloss` должен быть меньше `0.20`, `accuracy` больше `0.95`. Если это не так, то скорее всего у вас ошибка.

Теперь попробуем улучшить модель, добавив новые признаки, порождаемые словами. В `vowpal wabbit` есть возможность делать это прямо на лету. Воспользуйтесь параметрами `affix`, `ngram`, `skips`.

Далее везде при подборе параметров ориентируйтесь на улучшение `logloss`. Используйте `--quiet` или `-P`, чтобы избавиться от длинных выводов при обучении и применении моделей.

In [126]:
! vw -d train --affix -2t --loss_function logistic --oaa 5 -f model 2> trash
! vw -i model -t test -r pred 2> trash
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())
! vw -d train --affix -3t --loss_function logistic --oaa 5 -f model 2> trash
! vw -i model -t test -r pred 2> trash
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())
! vw -d train --affix -4t --loss_function logistic --oaa 5 -f model 2> trash
! vw -i model -t test -r pred 2> trash
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())
! vw -d train --affix -5t --loss_function logistic --oaa 5 -f model 2> trash
! vw -i model -t test -r pred 2> trash
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.14177
accuracy = 0.97814
logloss  = 0.14537
accuracy = 0.97799
logloss  = 0.14848
accuracy = 0.97780
logloss  = 0.15056
accuracy = 0.97762


Имеет смысл рассмотреть суффиксы длины 2, но мы не будем, так как изменение совсем незначительное.

In [127]:
for i in range(1, 3):
    ! vw -d train --affix "$i"t --loss_function logistic --oaa 5 -f model 2> trash
    ! vw -i model -t test -r pred 2> trash
    print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.13947
accuracy = 0.97818
logloss  = 0.14094
accuracy = 0.97817


Кажется, префиксы чуть более полезны, но тоже не то что бы оч сильно.

In [129]:
for i in range(2, 4):
    ! vw -d train --ngram t"$i" --loss_function logistic --oaa 5 -f model 2> trash
    ! vw -i model -t test -r pred 2> trash
    print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.17199
accuracy = 0.97599
logloss  = 0.18143
accuracy = 0.97571


Мда, в данном случае нграммы - зло.

Часто качество `vw` модели получается учушить увеличением числа проходов по обучающей выборке (параметр `--passes`) и увеличением числа бит хэш-функции для уменьшения числа коллизий признаков (параметр `-b`). Подробнее про то, где в `vowpal wabbit` используется хэш-функция, можно прочитать [здесь](https://github.com/JohnLangford/vowpal_wabbit/wiki/Feature-Hashing-and-Extraction). Как меняется качество при изменении этих параметров? Верно ли, что при увеличении значений параметров `--passes` и `-b` качество всегда не убывает и почему?

In [144]:
logloss = [0 for _ in range(10)]

In [145]:
for i in range(1, 5):
    ! vw -d train --passes "$i" --cache_file cache --loss_function logistic --oaa 5 -f model 2> trash
    ! vw -i model -t test -r pred 2> trash
    ll, acc = get_scores()
    logloss[i] = ll
    print('logloss  = %.5f\naccuracy = %.5f' % (ll, acc))

logloss  = 0.14763
accuracy = 0.97785
logloss  = 0.14696
accuracy = 0.97794
logloss  = 0.14722
accuracy = 0.97798
logloss  = 0.14845
accuracy = 0.97792


Нет смысла пытаться делать больше passes. Потому что видно, что если 2 прохода имеют какой-то смысл, то не больше.

Идеи, почему не улучшает:
    1. переобучение?
    2. порядок прохода не меняется, тем самым мы по сути повторяем, что уже было.

Давайте попробуем поизучать коллизии признаков, когда их хоть немного больше. Например, добавим префиксы длины 1. Единственное, что что-то хоть сколько-то существенно улучшало.

In [148]:
for i in range(19, 22):
    ! vw -d train --affix 1t -b "$i" --loss_function logistic --oaa 5 -f model 2> trash
    ! vw -i model -t test -r pred 2> trash
    print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

logloss  = 0.13925
accuracy = 0.97817
logloss  = 0.13934
accuracy = 0.97816
logloss  = 0.13957
accuracy = 0.97815


Только при малом увеличении происходит маленькое улучшение. Я думаю, что тут разгадка ясна - коллизий наверное почти и нет. И так как зависимость тут экспоненуциальная, то 1 байт решает проблему.

Теперь интерес представляет то, какие признаки оказались наиболее важными для модели. Для этого сначала переведем модель в читаемый формат.

In [149]:
! vw -i model -t --invert_hash model.readable train --quiet
! head -n 30 model.readable

Version 8.4.0
Id 
Min label:-50
Max label:50
bits:21
lda:0
0 ngram:
0 skip:
options: --affix 1t --oaa 5
Checksum: 1358628434
:0
Constant:928480:-2.87754
Constant[1]:928481:-3.50371
Constant[2]:928482:-3.2816
Constant[3]:928483:2.32285
Constant[4]:928484:-3.60427
affix^t+1=0:650976:0.0727394
affix^t+1=0[1]:650977:-0.47062
affix^t+1=0[2]:650978:-0.342807
affix^t+1=0[3]:650979:0.0704274
affix^t+1=0[4]:650980:-0.322214
affix^t+1=1:256320:0.684885
affix^t+1=1[1]:256321:-0.0287196
affix^t+1=1[2]:256322:0.736937
affix^t+1=1[3]:256323:-0.423031
affix^t+1=1[4]:256324:0.512856
affix^t+1=2:1958816:0.477723
affix^t+1=2[1]:1958817:0.0749873
affix^t+1=2[2]:1958818:0.679425
affix^t+1=2[3]:1958819:-0.365776


Первые несколько строк соответствуют информации о модели. Далее следуют строчки вида `feature[label]:hash:weight`. Выделите для каждого класса 10 признаков с наибольшими по модулю весами. Постарайтесь сделать ваш алгоритм прохода по файлу константным по памяти. Например, можно воспользоваться [кучей](https://docs.python.org/2/library/heapq.html).

Алгоритм с константной памятью:
    храним кучу для класса, в которой <= 10 элементов. И это ровно те фичи, которым соответствуют максимальные веса. В корне будет минимальный вес из этих 10. Когда нам приходит новый вес, то мы смотрим больше ли он минимального из этих 10, если да - выкидываем минимальный и добавляем новую фичу в кучу, иначе оставляем все как есть. Тогда в итоговой куче будут лужать требуемые фичи (ну или веса, а фичи будем отдельно хранить в сете), не суть.

In [168]:
import heapq
import codecs

In [178]:
heaps = [[] for _ in range(5)]
valuable_features = [dict() for _ in range(5)]
with open("model.readable") as f:
    index = 0
    for row in f:
        if index > 10:
            feature, h, weight = row[:-1].split(':')
            weight = float(weight)
            feature_name = feature
            label = 0
            if feature[-1] == ']':  
                feature_name = feature[:-3]
                label = int(feature[-2])
                
            if len(heaps[label]) < 10:
                heapq.heappush(heaps[label], weight)
                valuable_features[label][feature_name] = weight
            else:
                if heaps[label][0] < weight:
                    popped_w = heapq.heappop(heaps[label])
                    # like this because there can be repeated weights
                    for key, v in valuable_features[label].items():
                        if v == popped_w:
                            del valuable_features[label][key]
                            break
                    heapq.heappush(heaps[label], weight)
                    valuable_features[label][feature_name] = weight
        index = index + 1

In [179]:
print(valuable_features)

[{'affix^t+1=s': 0.813683, 'affix^t+1=c': 0.786124, 't^columnvalue': 0.747511, 'affix^t+1=j': 0.711068, 't^cexecute': 0.786124, 'affix^t+1=w': 0.75395, 'affix^t+1=r': 0.83535, 'affix^t+1=e': 0.741504, 'affix^t+1=p': 0.892719, 'affix^t+1=d': 0.747511}, {'t^interview': 1.46757, 't^commonparameters': 2.03654, 't^best': 1.26148, 't^tricks': 1.07165, 't^twitt': 1.07165, 't^viewed': 1.26148, 't^quadword': 2.03654, 't^books': 2.03654, 't^acceptcontact': 1.46757, 't^learn': 1.11343}, {'t^cumsum': 1.10484, 't^controlscode': 1.02274, 't^ubuntu': 1.52074, 't^ninessix': 1.52074, 't^programmers': 0.783551, 't^latex': 1.02274, 't^seo': 0.85912, 't^uscanada': 0.989284, 't^ssh': 0.989284, 't^developer': 1.10484}, {'t^firing': 2.69258, 't^intstr': 2.67892, 't^uiwebview': 2.62544, 't^noninfrastructure': 2.77701, 't^jscrollpane': 2.77701, 't^unhandeled': 2.62544, 't^classpath': 2.8423, 't^javar': 2.8423, 't^yampaglut': 2.67892, 't^viewmodel': 2.67892}, {'affix^t+1=s': 0.718723, 'affix^t+1=c': 0.704568, '

забавно, можно сказать, что 

0 - что-то непонятное

1 - интервью, советы

2 - что-то... более системное? общее

3 - как-то UI java в общем похоже на названия классов

4 - опять что-то непонятное =(

'not a real question',
'not constructive',
'off topic',
'open',
'too localized'
    
Похоже на правду! =)

Добавим признаки, извлеченные из текста поста (поле `BodyMarkdown`). В этом поле находится более подробная информация о вопросе, и часто туда помещают код, формулы и т.д. При удалении пунктуации мы потеряем много полезной информации, однако модель "мешка слов" на сырых данных может сильно раздуть признаковое пространство. В таких случаях работают с n-граммами на символах. <br>
Будьте осторожны: символы "`:`" и "`|`" нельзя использовать в названиях признаков, поскольку они являются служебными для `vw`-формата. Замените эти символы на два других редко встречающихся в выборке (или вообще не встречающихся). Также не забудьте про "`\n`". <br>
Поскольку для каждого документа одна n-грамма может встретиться далеко не один раз, то будет экономнее записывать признаки в формате `[n-грамма]:[число вхождений]`.

Также добавьте тэги (поля вид `TagN`). Придумайте и извлеките признаки из других полей. Только не используйте `PostClosedDate` – в нем содержится информация о таргете.

In [186]:
from collections import defaultdict

In [206]:
def get_ngrams(symbols, ngram=3):
    n = len(symbols)
    counter = defaultdict(int)
    for i in range(0, n - ngram + 1):
        new_ngram = ""
        for j in range(ngram):
            new_ngram = new_ngram + symbols[i + j]
        counter[new_ngram] += 1
        
    ngrams_str = ""
    for key, value in counter.items():
        ngrams_str = ngrams_str + str(key) + ":" + str(value) + " "
    return ngrams_str

def extract_ngram_body(row, ngram=3):
    text = row['BodyMarkdown']
    text = text.replace(":", "ǂ")
    text = text.replace("|", "ɀ")
    text = text.replace("\n", "ȸ")
    return get_ngrams(text, ngram)

def extract_tags(row):
    tags = ""
    for i in range(5):
        if len(row['Tag' + str(i + 1)]) == 0:
            pass
        else:
            tags = tags + row['Tag' + str(i + 1)] + " "
    return tags

Объединим все вместе. Реализуйте экстрактор признаков, который выделяет каждую подгруппу в отдельный namespace.

In [207]:
extractors_list = [
    ('t', extract_title), 
    ('b', extract_ngram_body), 
    ('a', extract_tags)
] # (namespace, extractor)


def make_feature_extractor(extractors_list):
    def feature_extractor(row):
        d = dict()
        for namespace, extractor in extractors_list:
            d[namespace] = extractor(row)
        return d
    return feature_extractor
    
data2vw(make_feature_extractor(extractors_list))

! head -n 1 train


4 |t decimal vs double |a c#  |b nd :1 oes:1 or :2 alu:1 in :1 'de:1  wa:1 uil:1 s o:1 al :1 e c:1 s a:1 rro:1 the:3 'm :1  fi:1 orȸ:1 ici:1 dou:2  ma:1 o ':1 tly:1 dec:2 onv:1 w t:1 r t:1  is:1 pac:2 ode:2 orm:1 *ȸȸ:1 tri:1 doe:1 ntr:1 k. :1 nsȸ:1 hen:2 m n:1 wor:2 s =:1 ert:1 y c:2 ȸȸW:1 00ȸ:1 Thi:2  ge:1  me:1 ans:3 ol :1 mal:2 o b:1 ld :1 ine:1 cim:2 . A:1  ty:1 tio:1 aci:2 VB.:1 tro:1 y =:1  bu:2 lue:1 itl:1 ed :2 B.N:1 sȸȸ:1 **C:1  tr:7 I g:1 ild:1 aki:1 kin:1 pli:1 I'm:1 opa:1 e w:1 ȸȸI:1 rol:1 bui:1 sn':1 ges:1 *Ca:1 is :4 ȸȸ*:1 NET:1  to:4 me :1 sug:1 est:1 wan:1 it,:1  my:1 ȸWh:1  I :3 rac:2 cod:2 ror:1 e f:2 his:4  su:1 , I:1  im:1   t:1 se :1 ked:1 **ȸ:1 he :2 lic:1 e**:1 000:1 tra:5 ȸȸ :1  a :2 s e:1  co:4 ann:1 y t:1 le,:1 ȸ**:1 to :4 cit:3 rt :1 t, :1 imp:1 thi:2 'do:1 d f:1  wo:2  = :2  do:2 't :1 o C:1 = t:2 try:1  Th:1 en :2  C#:1 ng :1 . T:1 ly :1 d m:1  'd:2 n't:1 #, :1 oub:2 500:1 d i:1    :4 r f:1 ran:3 ut :1 ET.:1 I t:2 n t:1 .Va:1 al':1 o u:1 eci:2 ant:1 ne :1 

In [208]:
! vw -d train --loss_function logistic --oaa 5 -f model
! vw -i model -t test -r pred
print('logloss  = %.5f\naccuracy = %.5f' % get_scores())

final_regressor = model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = train
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4        1      390
0.500000 0.000000            2            2.0        4        4      412
0.250000 0.000000            4            4.0        4        4      135
0.125000 0.000000            8            8.0        4        4      551
0.062500 0.000000           16           16.0        4        4      447
0.062500 0.062500           32           32.0        4        4      514
0.046875 0.031250           64           64.0        4        4      798
0.062500 0.078125          128          128.0        4        4      377
0.089844 0.117188          256          256.0        4        4      419
0.070312 0.050781          512          512.0   

Поэкспериментируйте с другими параметрами модели. Добавьте квадратичные взаимодействия между различными блоками признаков, измените параметры оптимизатора, добавьте регуляризацию и т.д. <br>
Выберите не менее трех параметров (и дискретные, и непрерывные). Для каждого из них объясните, почему по вашему мнению его изменение может улучшить качество модели, подберите оптимальное значение. Можете перебрать несколько значений в цикле, но лучше воспользоваться [vw-hypersearch](https://github.com/JohnLangford/vowpal_wabbit/wiki/Using-vw-hypersearch) или [vw-hyperopt](https://github.com/JohnLangford/vowpal_wabbit/blob/master/utl/vw-hyperopt.py) ([статья на хабре](https://habrahabr.ru/company/dca/blog/272697/)). Удобночитаемый вывод и/или графики для разных значений параметров обязательны.

Совсем необязательно добиваться наилучшего качества. Для нас важнее то, насколько хорошо вы разобрались с возможностями библиотеки и продемонстрировали это.


Помогло ли добавление квадратичных взаимодействий? Каких групп признаков? Какие параметры повлияли на улучшение качества сильнее всего?

In [ ]:
# YOUR CODE HERE

Приблизительная разбалловка:
* 3 балла в верно реализованные функции
* 1 балл за ответ на вопрос про параметры `--passes` и `-b` (с графиком/выводом)
* 2 балла за эффективное нахождение наиболее значимых признаков
* 1 балл за добавление своих признаков
* 3 балла за финальную часть (квадратичные взаимодействия, подбор параметров, графики/вывод и комментарии)